In [1]:
cd ../src

/Users/williamferreira/Dropbox/mscproject/src


In [2]:
import numpy as np
import pandas as pd

from model.utils import get_dataset, split_data

In [3]:
training_data = get_dataset('url-versions-2015-06-14-clean-train.csv')
X, y = split_data(training_data)

In [4]:
from model.classifiers.lr_predictors import LogitPredictor

In [5]:
from model.baseline.transforms import (
    RefutingWordsTransform,
    QuestionMarkTransform,
    HedgingWordsTransform,
    InteractionTransform,
    NegationOfRefutingWordsTransform,
    BoWTransform
)

from model.ext.transforms import (
    AlignedPPDBSemanticTransform,
    NegationAlignmentTransform,
    Word2VecSimilaritySemanticTransform,
    DependencyRootDepthTransform,
    SVOTransform,
)

In [8]:
predictor = LogitPredictor([QuestionMarkTransform,
                            BoWTransform,
                            Word2VecSimilaritySemanticTransform,
                            AlignedPPDBSemanticTransform,
                            NegationAlignmentTransform,
                            DependencyRootDepthTransform,
                            SVOTransform])

In [9]:
predictor.fit(X, y)

model/ext/transforms.py:188: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if e in SVOTransform._entailment_map.keys() and x == w]


LogitPredictor(transforms=[<class 'model.baseline.transforms.QuestionMarkTransform'>, <class 'model.baseline.transforms.BoWTransform'>, <class 'model.ext.transforms.Word2VecSimilaritySemanticTransform'>, <class 'model.ext.transforms.AlignedPPDBSemanticTransform'>, <class 'model.ext.transforms.NegationAlignmentTransform'>, <class 'model.ext.transforms.DependencyRootDepthTransform'>, <class 'model.ext.transforms.SVOTransform'>])

In [10]:
test_data = get_dataset('url-versions-2015-06-14-clean-test.csv')
X, y = split_data(test_data)

In [11]:
example = X[X.articleId == '18eea9c0-47e3-11e4-9960-a165dd517bbb']

In [12]:
predictor.predict_proba(example)

array([[ 0.68750213,  0.30108751,  0.01141035]])

In [12]:
y_hat = predictor.predict(X)

In [13]:
Z = X.copy()
Z['y'] = y
Z['y_hat'] = y_hat

In [13]:
clean_test_errors = Z[(Z.y != Z.y_hat)][['claimHeadline', 'articleHeadline', 'y', 'y_hat', 'articleId', 'claimId']]

In [15]:
clean_test_errors[clean_test_errors.y_hat == 'against'].shape

(20, 6)

In [14]:
clean_test_errors[clean_test_errors.articleId == '60669e60-c106-11e4-8747-5b024dd6dd6c']

,claimHeadline,articleHeadline,y,y_hat,articleId,claimId
2438,Tiger Woods is serving a suspension from the P...,"Tiger Woods PED, PGA Tour Suspension Allegatio...",against,for,60669e60-c106-11e4-8747-5b024dd6dd6c,10a1faa0-c0fc-11e4-8747-5b024dd6dd6c


In [14]:
Z[Z.y == Z.y_hat][['claimHeadline', 'articleHeadline', 'y', 'y_hat', 'articleId', 'claimId']].to_csv('../output/url-versions-2015-06-14-clean-test-correct.csv')
Z[Z.y != Z.y_hat][['claimHeadline', 'articleHeadline', 'y', 'y_hat', 'articleId', 'claimId']].to_csv('../output/url-versions-2015-06-14-clean-test-errors.csv')

In [12]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [13]:
accuracy_score(y, y_hat)

0.70997123681687435

In [17]:
pd.Series(y).value_counts()

for          1019
observing     808
against       343
dtype: int64

In [18]:
y.shape

(2170,)

In [19]:
pd.Series(y).value_counts() / pd.Series(y).value_counts().sum()

for          0.469585
observing    0.372350
against      0.158065
dtype: float64